# 分析工具类

负责使用递归根据指定最终产物及其数量分析所需基本材料及其数量，并进行时间对齐，即量化生产过程。

生产效率统一以分钟为准。

暴露以下API：

- 根据产物及数量，计算材料及数量，可以指定是否使用高效配方

In [9]:
import Ipynb_importer
import datautil

import pandas as pd

from IPython.display import display


class AnalysisUtil:
    # 生产单元清单，列出所有的生产单元及其数量
    unit_lst = []

    # 单条产线的生产效率是一定的，如果需要更多的数量，只能水平扩展更多的产线
    # 所以根据最终的需求，比如一分钟需要生产100个某产品，只能计算出“需要几条产线”
    # 从配方树的递归中，能统计出单条产线一分钟的产出数
    # 问题模型：假设材料充盈，要达到目标产量，需要多少生产单元？
    # need就是需求的每分钟产量，在上级产品中计算出需要下级材料的need，然后传入递归

    @classmethod
    def make_formula_tree(cls, name, need, efficient_enabled=False, adjustable_enabled=False, times=1.0, deep=0, pid=0):
        # 查询配方
        # 如果启用了高效，先查找是否有高效配方
        formula = datautil.query_formula_by_product_name(
            name, efficient_enabled)
        if formula is None or not formula.is_enabled:
            # 如果查不到高效配方，或者高效配方未启用，降级为普通配方
            formula = datautil.query_formula_by_product_name(name)
        # 主产品名
        product_name = formula.product.split(',')[0].split('-')[0]
        # 主产品单元产量
        product_unit_output = formula.product.split(',')[0].split('-')[1]
        # 如果启用了可变速度，检查该配方是否可变速，只有制造台配方可变速
        if not adjustable_enabled or not formula.is_adjustable:
            # 如果两个条件任一个不成立
            # 将生产速度置为1
            times = 1.0
        # 计算每分钟产量的公式
        # 如果启用了可变速度，则相同时间内，单元产量=标准产量*速度倍数，再除以秒数就是变速后的每秒产量
        product_unit_output_m = (int(product_unit_output)
                                 * times/formula.work_time)*60
        # 生产单元数=需求/每分钟产量
        unit = need/product_unit_output_m

        # 递归深度递增
        deep += 1
        # 向清单中添加一个字典，后续使用pandas展示
        cls.unit_lst.append({'deep': deep, 'id': formula.id, 'pid': pid, '产品名': product_name,
                             '每分钟需求': need, '生产单元数': unit})

        # 获取所有的材料列表
        materials = [(i.split('-')[0], i.split('-')[1])
                     for i in formula.material.split(',')]
        for i, m in enumerate(materials):
            # 材料名
            material_name = m[0]
            # 每个生产单元需要的该种材料的数量
            material_unit_need = m[1]
            # 计算产出比，即产品单元产量除以材料单元需求
            # ratio意为比例
            ratio = int(product_unit_output)/int(material_unit_need)
            # 材料每分钟需求=产品每分钟产量/产出比，
            # 因为 产品每分钟产量/材料每分钟需求=产品单元产量/材料单元需求，
            # 即在单元生产的基础上乘以相同时间
            material_unit_need_m = product_unit_output_m/ratio
            # 材料每分钟需求数=材料每单元每分钟需求数*单元数
            material_need_m = material_unit_need_m*unit

            # 查询该种材料
            item = datautil.query_one_item(material_name)
            if item.level == 1:
                # 如果是原材料等级，打断递归
                # 并向清单中添加字典
                cls.unit_lst.append({'deep': deep+1, 'id': -1, 'pid': formula.id, '产品名': material_name,
                                     '每分钟需求': material_need_m, '生产单元数': -1})
                continue
            else:
                # 如果不是原材料等级，进行递归
                cls.make_formula_tree(material_name, material_need_m,
                                      efficient_enabled=efficient_enabled, adjustable_enabled=adjustable_enabled,
                                      times=times, deep=deep, pid=formula.id)

    @ classmethod
    def get_tree(cls, name, need, efficient_enabled=False, adjustable_enabled=False, times=1.0):
        cls.unit_lst.clear()
        cls.make_formula_tree(name, need, efficient_enabled=efficient_enabled,
                              adjustable_enabled=adjustable_enabled, times=times)
        df = pd.DataFrame(cls.unit_lst)
#         df.sort_values(by='level')
        return df


df = AnalysisUtil.get_tree(
    '太阳帆', 1200, efficient_enabled=False, adjustable_enabled=True, times=1.0)
df.sort_values(by='pid')

,deep,id,pid,产品名,每分钟需求,生产单元数
0,1,1,0,太阳帆,1200.0,40.0
1,2,2,1,石墨烯,600.0,15.0
9,2,3,1,光子合并器,600.0,30.0
2,3,9,2,高能石墨,900.0,30.0
4,3,10,2,硫酸,300.0,7.5
13,3,4,3,电路板,600.0,5.0
10,3,5,3,棱镜,1200.0,20.0
14,4,7,4,铁块,600.0,10.0
16,4,8,4,铜块,300.0,5.0
11,4,6,5,玻璃,1800.0,60.0
